In [1]:
PATH_TO_DATA="/Users/roman/Data_Saints/data/X5/"
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.cluster import k_means
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import pairwise_distances

### Формирование dataFame

In [2]:
data=pd.read_csv(PATH_TO_DATA+"X5.csv",sep=";")
data.head()

,USER_ID,DT,PLU,QTY,ORDER_ID
0,1,03.06.16,3456173,1,1
1,1,03.06.16,6178,"0,24",1
2,1,03.06.16,3456172,1,1
3,1,03.06.16,3274874,"0,85",1
4,1,03.06.16,3270705,2,1


In [ ]:
#Явных пропусков нет(хотя возможно есть заглушка)
# data.isnull().any()

In [ ]:
#data.info()

In [ ]:
#На сервере считалось

# # Изменяю тип данных
# data["DT"]=pd.to_datetime(data["DT"])
# data["QTY"]=data["QTY"].apply(lambda x:x.replace(",","."))
# data["QTY"]=data["QTY"].astype(float)

In [ ]:
#На сервере считалось

# data_OHE=pd.concat([data,pd.get_dummies(data.PLU,prefix="PLU")],axis=1)
# data_OHE.drop("PLU",axis=1,inplace=True)
# data_OHE.head()

In [3]:
#Часть данных
data_part=data[:1000000]

In [4]:
enc = OneHotEncoder()
tmp=data_part.PLU.as_matrix()
a=enc.fit_transform(tmp.reshape(-1,1))

In [ ]:
# Изменяю тип данных
# data_part["DT"]=pd.to_datetime(data_part["DT"])
# data_part["QTY"]=data_part["QTY"].apply(lambda x:x.replace(",","."))
# data_part["QTY"]=data_part["QTY"].astype(float)

data_part.drop(["DT","QTY"],axis=1,inplace=True)

In [ ]:
data_p_OHE=pd.concat([data_part,pd.get_dummies(data_part.PLU,prefix="PLU")],axis=1)
data_p_OHE.drop("PLU",axis=1,inplace=True)
data_p_OHE.head()

In [ ]:
# Столбец QTY преобразую в 2 столбца:
# кол-во штук
# вес
data_p_OHE["num"]=data_p_OHE["QTY"].apply(lambda x:x if x%1==0 else 0)
data_p_OHE["weight"]=data_p_OHE["QTY"].apply(lambda x:x if x%1!=0 else 0)
data_p_OHE.drop("QTY",axis=1,inplace=True)
data_p_OHE.head()

In [ ]:
#Удаляю дату кол-во тввара и вес товара чтобы схлопнуть все для сегментации
data_p_OHE.drop(["DT","num","weight"],axis=1,inplace=True)

In [ ]:
data_p_OHE.head()

In [ ]:
# Кол-во посещений магазина у каждого пользователя
tmp=data_p_OHE[["USER_ID","ORDER_ID"]]
num_purc=pd.pivot_table(tmp,index="USER_ID",aggfunc="max")["ORDER_ID"]

In [ ]:
#Складываю все покупки для каждлго пользователя
df=pd.pivot_table(data_p_OHE,index=["USER_ID"],aggfunc=np.sum)
df["ORDER_ID"]=num_purc

In [ ]:
df.head()

In [ ]:
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist

# X = np.zeros((150, 2))
df=a

# distance_mat = pdist(df) # pdist посчитает нам верхний треугольник матрицы попарных расстояний
distance_mat=pairwise_distances(a,n_jobs=-1)

In [ ]:
a.shape

In [17]:
pairwise([1,2,3])

TypeError: 'module' object is not callable

In [ ]:
Z = hierarchy.linkage(distance_mat, 'single') # linkage — реализация агломеративного алгоритма
plt.figure(figsize=(10, 5))
dn = hierarchy.dendrogram(Z)

In [ ]:
a=k_means(df,n_clusters=2)

In [ ]:
a[1]

In [ ]:
metrics.silhouette_score(df, a[1])

In [ ]:
from sklearn import metrics
from sklearn import datasets
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation, SpectralClustering

X=a
for n_cluster in range(2,11):
    algorithms = []
    algorithms.append(KMeans(n_clusters=n_cluster, random_state=1))

    algorithms.append(AgglomerativeClustering(n_clusters=n_cluster))
    
    data = []
    for algo in algorithms:
        algo.fit(X)
        data.append(({'Silhouette': metrics.silhouette_score(X, algo.labels_)}))
    
    results = pd.DataFrame(data=data, columns=['Silhouette'],
                           index=['K-means', 'Agglomerative'])
    
    print("n_cluster=%s"%n_cluster)
    print(results)
    print()